In [1]:
import random
import numpy as np
from tqdm import tqdm

random.seed(2000)

In [2]:
NUM_ITERS = 10000  # Number of iterations for a loop or a process.

ENRICHMENT_RANGE = np.linspace(0, 100, 10000)  # Creating an array of 10,000 evenly spaced values from 0 to 100 using numpy.
# This range represent possible values for enrichment parameter

In [3]:
class Country():
    def __init__(self, name, acceptables, proposals):
        self.name = name  # Assigning the name of the country.
        self.acceptables = acceptables  # Storing the acceptable demands for the country.
        self.proposals = proposals  # Storing the proposals for the country.
        
    def propose(self, iter):
        # Method for proposing a random proposal based on the iteration number.
        for proposal in self.proposals:
            if iter in proposal['valid']:
                random_proposal = {demand: random.choice(proposal[demand]) for demand in self.acceptables[0].keys()}
                # Randomly selecting a value for each demand from the proposal's valid range.
                return random_proposal
            
    def evaluate(self, proposal):
        # Method for evaluating the proposal against the country's acceptable demands.
        for acceptable in self.acceptables:
            acceptance = []
            for demand in self.acceptables[0].keys():
                # Checking if the demand value falls within the acceptable range for each demand.
                dem_value = proposal[demand]
                acc_range = acceptable[demand]
                acceptance.append(dem_value in acc_range)
            
            if all(acceptance):
                return True  # If all demands are accepted, return True.
        
        return False  # If any demand is not accepted, return False.


In [4]:
# Iran has one set of acceptable demands. 
# The 'enrichment' demand should have values greater than 60, 
# 'guns' can be either True or False, don't care about whether it needs to send guns to russia
# and 'not_support' can be either True or False, don't care about us supporting 
iran_acceptables = [
    {
        'enrichment': ENRICHMENT_RANGE[ENRICHMENT_RANGE > 60].tolist(),
        'guns': [True, False],
        'not_support': [True, False]
    }
]

us_acceptables = [
    {
        # The US has two sets of acceptable demands. 
        # The first set requires 'enrichment' values less than 20, 
        # 'guns' can be either True or False, they are happy as long as enrichment is below 20
        # 'not_support' can be either True or False, they are happy as long as enrichment is below 20
        'enrichment': ENRICHMENT_RANGE[ENRICHMENT_RANGE < 20].tolist(),
        'guns': [True, False],
        'not_support': [True, False]
    },
    {
        # The second set requires 'enrichment' values less than 80, 
        # 'guns' should be False, if Iran is allowed to enrich upto 80, then it should not send guns to russia
        # 'not_support' can be either True or False, do not care about supporting
        'enrichment': ENRICHMENT_RANGE[ENRICHMENT_RANGE < 80].tolist(),
        'guns': [False],
        'not_support': [True, False]
    }
]

russia_acceptables = [
        # Russia has two sets of acceptable demands. 
        # Both sets accept any 'enrichment' value, 
    {
        # but the first set requires 'guns' to be True and 
        # 'not_support' can be either True or False, 
        # this means that, as long as russia recieves its guns, they do not care about US supporting
        'enrichment': ENRICHMENT_RANGE.tolist(),
        'guns': [True],
        'not_support': [True, False]
    },
    { 
        # the second set requires 'guns' to be False and 
        # 'not_support' to be False.
        # if russia can not get any guns from Iran, the US can not support Ukraine
        'enrichment': ENRICHMENT_RANGE.tolist(),
        'guns': [False],
        'not_support': [True]
    }
]

In [5]:
iran_proposals = [
    {
        # Iran has one proposal. It is valid for all iteration numbers
        # It specifies 'enrichment' values greater than 60, 
        # 'guns' can be either True or False, don't care about supporting russia
        # 'not_support' can be either True or False, don't care about US supporting Ukraine
        'valid': list(range(NUM_ITERS)),
        'enrichment': ENRICHMENT_RANGE[ENRICHMENT_RANGE > 60].tolist(),
        'guns': [True, False],
        'not_support': [True, False]
    }
]

us_proposals = [
    {
        # The US has three proposals. 
        # The first proposal is valid for the first five iteration numbers.
        # It specifies 'enrichment' values less than 20, 
        # 'guns' can be either True or False,
        # 'not_support' can be either True or False. 
        # as long as enrichment is below 20, they do not care about sending guns to ukraine or iran sending guns to russia
        'valid': list(range(5)),
        'enrichment': ENRICHMENT_RANGE[ENRICHMENT_RANGE < 20].tolist(),
        'guns': [True, False],
        'not_support': [True, False]
    },
    { 
        # The second proposal is valid for iteration numbers 5 to 6.
        # specifies 'enrichment' values between 20 and 80 (exclusive)
        # 'guns' can be either True or False,
        # 'not_support' can be either True or False. 
        # for a short period during the negotiation they may accept enrichment between 20 to 80, without any extra conditions
        'valid': list(range(5, 7)),
        'enrichment': ENRICHMENT_RANGE[(ENRICHMENT_RANGE > 20) & (ENRICHMENT_RANGE < 80)].tolist(),
        'guns': [True, False],
        'not_support': [True, False]
    },
    {
        # The third proposal is valid from iteration number 7 to the maximum number of iterations
        # It specifies 'enrichment' values between 20 and 80 (exclusive), 
        # 'guns' should be False,
        # 'not_support' can be either True or False.
        # after that short period, they only accept enrichment between 20 to 80 if and only if iran does not send guns to russia
        'valid': list(range(7, NUM_ITERS)),
        'enrichment': ENRICHMENT_RANGE[(ENRICHMENT_RANGE > 20) & (ENRICHMENT_RANGE < 80)].tolist(),
        'guns': [False],
        'not_support': [True, False]
    }
]

russia_proposals = [
    {
        # Russia has two proposals. 
        # All three proposals are valid for all iteration numbers and 
        # specify 'enrichment' values greater than 50, 
        'valid': list(range(NUM_ITERS)),
        'enrichment': ENRICHMENT_RANGE[ENRICHMENT_RANGE > 50].tolist(),
        'guns': [True], # if russia can recieve iranian guns, they do not care about us supporting
        'not_support': [True, False]
    },
    {
        'valid': list(range(NUM_ITERS)),
        'enrichment': ENRICHMENT_RANGE[ENRICHMENT_RANGE > 50].tolist(),
        'guns': [False], # if they can not recieve guns, us can not support 
        'not_support': [True]
    }
]

In [6]:
# Creates a Country object named 'iran' with the corresponding acceptable demands and proposals defined earlier for Iran.
iran = Country('iran', iran_acceptables, iran_proposals)
# Creates a Country object named 'us' with the corresponding acceptable demands and proposals defined earlier for the US.
us = Country('us', us_acceptables, us_proposals)
# Creates a Country object named 'russia' with the corresponding acceptable demands and proposals defined earlier for Russia.
russia = Country('russia', russia_acceptables, russia_proposals)
# All these country objects are then stored in a list called countries.
countries = [iran, us, russia]

In [7]:
agreement = False  # Initialize a variable to track if an agreement is reached.
agreed_terms = None  # Initialize a variable to store the agreed terms of the proposal.
proposed_terms = []  # Initialize an empty list to store the details of all proposed terms.

# Loop over iterations using tqdm to create a progress bar.
for iter in tqdm(range(NUM_ITERS)):
    proposals = {country.name: country.propose(iter) for country in countries}
    # Generate proposals for each country at the current iteration.
    
    for country, proposal in proposals.items():
        country_accepted = {country.name: country.evaluate(proposal) for country in countries}
        # Evaluate the proposal of each country against the acceptable demands of all countries.
        
        agreement = all(country_accepted.values())
        # Check if all countries have accepted the proposal.
        
        proposed_terms.append(f'Round {iter+1}: {country} -> {proposal} | votes: {country_accepted}')
        # Store the details of the proposed terms, including the round number, country, proposal, and acceptance votes.
        
        if agreement:
            agreed_terms = proposal
            break  # Exit the loop if an agreement is reached.
    
    if agreement:
        break  # Exit the outer loop if an agreement is reached.

  0%|          | 5/10000 [00:00<00:11, 861.61it/s]


In [8]:
for term in proposed_terms:
    print(term)

Round 1: iran -> {'enrichment': 78.36783678367837, 'guns': True, 'not_support': False} | votes: {'iran': True, 'us': False, 'russia': True}
Round 1: us -> {'enrichment': 19.401940194019403, 'guns': False, 'not_support': True} | votes: {'iran': False, 'us': True, 'russia': True}
Round 1: russia -> {'enrichment': 97.07970797079709, 'guns': True, 'not_support': False} | votes: {'iran': True, 'us': False, 'russia': True}
Round 2: iran -> {'enrichment': 67.6967696769677, 'guns': True, 'not_support': False} | votes: {'iran': True, 'us': False, 'russia': True}
Round 2: us -> {'enrichment': 7.040704070407041, 'guns': False, 'not_support': True} | votes: {'iran': False, 'us': True, 'russia': True}
Round 2: russia -> {'enrichment': 66.9066906690669, 'guns': True, 'not_support': False} | votes: {'iran': True, 'us': False, 'russia': True}
Round 3: iran -> {'enrichment': 67.93679367936794, 'guns': True, 'not_support': True} | votes: {'iran': True, 'us': False, 'russia': True}
Round 3: us -> {'enric


    Round 1:
        Iran proposes: {'enrichment': 78.36783678367837, 'guns': True, 'not_support': False}
        US proposes: {'enrichment': 19.401940194019403, 'guns': False, 'not_support': True}
        Russia proposes: {'enrichment': 97.07970797079709, 'guns': True, 'not_support': False}
        Votes:
            Iran: True (accepted)
            US: False (not accepted)
            Russia: True (accepted)

    Round 2:
        Iran proposes: {'enrichment': 67.6967696769677, 'guns': True, 'not_support': False}
        US proposes: {'enrichment': 7.040704070407041, 'guns': False, 'not_support': True}
        Russia proposes: {'enrichment': 66.9066906690669, 'guns': True, 'not_support': False}
        Votes:
            Iran: True (accepted)
            US: False (not accepted)
            Russia: True (accepted)

    Round 3:
        Iran proposes: {'enrichment': 67.93679367936794, 'guns': True, 'not_support': True}
        US proposes: {'enrichment': 19.411941194119414, 'guns': True, 'not_support': False}
        Russia proposes: {'enrichment': 59.25592559255926, 'guns': True, 'not_support': True}
        Votes:
            Iran: True (accepted)
            US: False (not accepted)
            Russia: True (accepted)

    Round 4:
        Iran proposes: {'enrichment': 71.57715771577158, 'guns': False, 'not_support': False}
        US proposes: {'enrichment': 1.08010801080108, 'guns': False, 'not_support': True}
        Russia proposes: {'enrichment': 66.12661266126614, 'guns': True, 'not_support': False}
        Votes:
            Iran: True (accepted)
            US: True (accepted)
            Russia: False (not accepted)

    Round 5:
        Iran proposes: {'enrichment': 85.25852585258527, 'guns': True, 'not_support': True}
        US proposes: {'enrichment': 5.7905790579057905, 'guns': True, 'not_support': False}
        Russia proposes: {'enrichment': 96.47964796479648, 'guns': True, 'not_support': True}
        Votes:
            Iran: True (accepted)
            US: False (not accepted)
            Russia: True (accepted)

    Round 6:
        Iran proposes: {'enrichment': 61.62616261626163, 'guns': True, 'not_support': True}
        US proposes: {'enrichment': 62.16621662166217, 'guns': False, 'not_support': True}
        Votes:
            Iran: True (accepted)
            US: True (accepted)
            Russia: True (accepted)


    In each round, the countries (Iran, US, Russia) propose their terms of the negotiation.
    For each proposal, the votes of all countries are shown, indicating whether each country accepts or rejects the proposal.
    The voting results are based on the evaluation of each country's proposal against their acceptable demands.
    A value of True in the votes means that the country accepts the proposal, while False means it does not accept.
    In the provided result, an agreement is reached in Round 6, where all countries (Iran, US, Russia) accept the proposal made by US: {'enrichment': 62.16621662166217, 'guns': False, 'not_support': True}.
    The negotiation process stops once an agreement is reached, and the loop is terminated.

In [9]:
print(iter+1)
print(agreed_terms)   

6
{'enrichment': 62.16621662166217, 'guns': False, 'not_support': True}
